In [1]:
from hail import Table
import os
import pprint
from pprint import pformat
import argparse
import hail as hl
import pandas as pd
import numpy as np
import pyspark
from pprint import pformat
import json
import sys
import re
from pathlib import Path
import logging
from typing import Any, Counter, List, Optional, Tuple, Union, Dict
import uuid
import json

tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
#plot_dir = "/home/ubuntu/data/tmp"
plot_dir="/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"
lustre_dir = "file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=lustre_dir, local_tmpdir=lustre_dir, default_reference="GRCh38")



pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/ubuntu/data/scripts/exomeQC-hail-gnomad/notebooks/hail-20210528-1024-0.2.62-84fa81b9ea3d.log


In [2]:
# ht from RF
run_hash="94f5cc00"

ht = hl.read_table(
        f'{lustre_dir}/variant_qc/models/{run_hash}/rf_result_sanger_cohorts_new.ht')




In [3]:
ht.describe()

----------------------------------------
Global fields:
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'a_index': int32 
    'was_split': bool 
    'AC': array<int32> 
    'AF': array<float64> 
    'AN': int32 
    'AS_BaseQRankSum': array<float64> 
    'AS_FS': array<float64> 
    'AS_InbreedingCoeff': array<float64> 
    'AS_MQ': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_QD': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'BaseQRankSum': float64 
    'DB': bool 
    'DP': int32 
    'DS': bool 
    'END': int32 
    'ExcessHet': float64 
    'FS': float64 
    'InbreedingCoeff': float32 
    '

In [4]:
synonymous="s3a://DDD-ELGH-UKBB-exomes/vep/vep_consequences_worst_synonymous.tsv"

In [5]:
synonymous

's3a://DDD-ELGH-UKBB-exomes/vep/vep_consequences_worst_synonymous.tsv'

In [6]:
ht_synonymous=hl.import_table(synonymous,impute=True, no_header=True)

2021-05-28 10:25:06 Hail: INFO: Reading table to impute column types
2021-05-28 10:25:14 Hail: INFO: Finished type imputation
  Loading field 'f0' as type str (imputed)
  Loading field 'f1' as type int32 (imputed)
  Loading field 'f2' as type str (imputed)
  Loading field 'f3' as type str (imputed)
  Loading field 'f4' as type str (imputed)


In [9]:
ht_synonymous.show()

,,,,
f0,f1,f2,f3,f4
str,int32,str,str,str
"""chr1""",69081,"""G""","""C""","""synonymous_variant"""
"""chr1""",69270,"""A""","""G""","""synonymous_variant"""
"""chr1""",69360,"""C""","""T""","""synonymous_variant"""
"""chr1""",69366,"""T""","""G""","""synonymous_variant"""
"""chr1""",69369,"""G""","""A""","""synonymous_variant"""
"""chr1""",69393,"""T""","""C""","""synonymous_variant"""
"""chr1""",69438,"""T""","""C""","""synonymous_variant"""
"""chr1""",69453,"""G""","""A""","""synonymous_variant"""


In [32]:
ht=hl.import_table(synonymous,types={'f0':'str','f1':'int32', 'f2':'str','f3':'str','f4':'str'}, no_header=True)
ht=ht.annotate(chr=ht.f0)
ht=ht.annotate(pos=ht.f1)
ht=ht.annotate(ref=ht.f2)
ht=ht.annotate(alt=ht.f3)
ht=ht.annotate(consequence=ht.f4)
#ht=ht.select(ht.chrom,ht.position)
ht = ht.key_by(
    locus=hl.locus(ht.chr, ht.pos), alleles=[ht.ref,ht.alt] , reference_genome='GRCh38')


2021-05-28 10:55:09 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type str (user-supplied)
  Loading field 'f3' as type str (user-supplied)
  Loading field 'f4' as type str (user-supplied)


In [33]:
ht.show()

,,,,,,,,,,,,
f0,f1,f2,f3,f4,chr,pos,ref,alt,consequence,locus,alleles,reference_genome
str,int32,str,str,str,str,int32,str,str,str,locus<GRCh38>,array<str>,str
"""chr1""",69081,"""G""","""C""","""synonymous_variant""","""chr1""",69081,"""G""","""C""","""synonymous_variant""",chr1:69081,"[""G"",""C""]","""GRCh38"""
"""chr1""",69270,"""A""","""G""","""synonymous_variant""","""chr1""",69270,"""A""","""G""","""synonymous_variant""",chr1:69270,"[""A"",""G""]","""GRCh38"""
"""chr1""",69360,"""C""","""T""","""synonymous_variant""","""chr1""",69360,"""C""","""T""","""synonymous_variant""",chr1:69360,"[""C"",""T""]","""GRCh38"""
"""chr1""",69366,"""T""","""G""","""synonymous_variant""","""chr1""",69366,"""T""","""G""","""synonymous_variant""",chr1:69366,"[""T"",""G""]","""GRCh38"""
"""chr1""",69369,"""G""","""A""","""synonymous_variant""","""chr1""",69369,"""G""","""A""","""synonymous_variant""",chr1:69369,"[""G"",""A""]","""GRCh38"""
"""chr1""",69393,"""T""","""C""","""synonymous_variant""","""chr1""",69393,"""T""","""C""","""synonymous_variant""",chr1:69393,"[""T"",""C""]","""GRCh38"""
"""chr1""",69438,"""T""","""C""","""synonymous_variant""","""chr1""",69438,"""T""","""C""","""synonymous_variant""",chr1:69438,"[""T"",""C""]","""GRCh38"""
"""chr1""",69453,"""G""","""A""","""synonymous_variant""","""chr1""",69453,"""G""","""A""","""synonymous_variant""",chr1:69453,"[""G"",""A""]","""GRCh38"""


In [34]:
ht=ht.drop(ht.f0,ht.f1,ht.f2,ht.f3,ht.f4,ht.chr,ht.pos,ht.ref,ht.alt)

In [20]:
ht.show()

,,,
consequence,locus,alleles,reference_genome
str,locus<GRCh38>,array<str>,str
"""synonymous_variant""",chr1:69081,"[""G"",""C""]","""GRCh38"""
"""synonymous_variant""",chr1:69270,"[""A"",""G""]","""GRCh38"""
"""synonymous_variant""",chr1:69360,"[""C"",""T""]","""GRCh38"""
"""synonymous_variant""",chr1:69366,"[""T"",""G""]","""GRCh38"""
"""synonymous_variant""",chr1:69369,"[""G"",""A""]","""GRCh38"""
"""synonymous_variant""",chr1:69393,"[""T"",""C""]","""GRCh38"""
"""synonymous_variant""",chr1:69438,"[""T"",""C""]","""GRCh38"""
"""synonymous_variant""",chr1:69453,"[""G"",""A""]","""GRCh38"""


In [35]:
ht = ht.key_by(
    ht.locus, ht.alleles)

In [36]:
ht.show()

,,,
consequence,locus,alleles,reference_genome
str,locus<GRCh38>,array<str>,str
"""synonymous_variant""",chr1:69081,"[""G"",""C""]","""GRCh38"""
"""synonymous_variant""",chr1:69270,"[""A"",""G""]","""GRCh38"""
"""synonymous_variant""",chr1:69360,"[""C"",""T""]","""GRCh38"""
"""synonymous_variant""",chr1:69366,"[""T"",""G""]","""GRCh38"""
"""synonymous_variant""",chr1:69369,"[""G"",""A""]","""GRCh38"""
"""synonymous_variant""",chr1:69393,"[""T"",""C""]","""GRCh38"""
"""synonymous_variant""",chr1:69438,"[""T"",""C""]","""GRCh38"""
"""synonymous_variant""",chr1:69453,"[""G"",""A""]","""GRCh38"""


In [37]:
ht.write(f'{lustre_dir}/grch38_synonymous_variants.ht', overwrite=True)

2021-05-28 10:55:57 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-28 10:56:18 Hail: INFO: wrote table with 1399620 rows in 1 partition to file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc/grch38_synonymous_variants.ht
    Total size: 20.68 MiB
    * Rows: 20.68 MiB
    * Globals: 11.00 B
    * Smallest partition: 1399620 rows (20.68 MiB)
    * Largest partition:  1399620 rows (20.68 MiB)


In [38]:
# ht from RF
run_hash="94f5cc00"

ht = hl.read_table(
        f'{lustre_dir}/variant_qc/models/{run_hash}/rf_result_sanger_cohorts_new.ht')

ht_synonymous=hl.read_table( f'{lustre_dir}/grch38_synonymous_variants.ht')

In [41]:
ht=ht.annotate(consequence=ht_synonymous[ht.key].consequence)

In [44]:
ht.consequence.summarize()

Non-missing,1234902 (4.41%)
Missing,26773141 (95.59%)
Min Size,18
Max Size,97
Mean Size,22.10
Sample Values,"['synonymous_variant', 'synonymous_variant', 'synonymous_variant', 'synonymous_variant', 'synonymous_variant']"


In [45]:
ht.write(f'{lustre_dir}/variant_qc/models/{run_hash}/rf_result_sanger_cohorts_new_SYNONYMOUS.ht', overwrite=True)

2021-05-28 10:59:00 Hail: INFO: wrote table with 28008043 rows in 487 partitions to file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc/variant_qc/models/94f5cc00/rf_result_sanger_cohorts_new_SYNONYMOUS.ht
    Total size: 3.05 GiB
    * Rows: 3.05 GiB
    * Globals: 292.00 B
    * Smallest partition: 10484 rows (1.23 MiB)
    * Largest partition:  59037 rows (7.00 MiB)
